Вводная часть и некоторые общие положения об интерпретаторах питона, информация по установке и настройке, отличиях CPython и PyPy3 изложены в ридми [репозитория](https://github.com/Asklepiad/PyPy_vs_CPython.git) . В ноутбуках будут показаны тесты на конкретных примерах, демонстрирующие отличия в логике и производительности двух интерпретаторов.

# 1. Скорость базового питона

## 1.1 Теоретическая посылка и базовый пример

Считается, что PyPy значительно ускоряет базовый питон. Проверим, даст ли он индульгиенцию на использование двойных циклов. Сравним ресурсоемкий примтивный код на питоне и в PyPy.

In [15]:
%%time
bad_naming = range(1000)
for i in bad_naming:
    for j in bad_naming:
        for k in bad_naming:
            i + j * k

CPU times: user 1min 21s, sys: 0 ns, total: 1min 21s
Wall time: 1min 21s


Тройной цикл исполнялся `1 минуту, 21 секунду`

## 1.2 Осмысленный пример

Попробуем что-нибудь более осмысленное, чем тройной цикл. Например, двойной цикл. Я взял свой код алгоритма Нидлмана-Вунша из первого семестра (опустим сейчас кривость и неоптимальность и посмотрим, как интепретаторы прожуют сие). Для сравнения я взял довольно длинные последовательности: по 10000 нуклеотидов каждый.

In [16]:
import random

In [17]:
%%time
x, y = ''.join(random.choices("AGCT", k=10000)), ''.join(random.choices("AGCT", k=10000))
n = len(x)
m = len(y)
mat = [[0 for j in range(m+1)] for i in range(n+1)]
for j in range(m+1):
    mat[0][j] = -j
for i in range(n+1):
    mat[i][0] = -i

for i in range(1, n+1):
    for j in range(1, m+1):
        if x[i-1] == y[j-1]:
            diag = mat[i-1][j-1]+1
        else:
            diag = mat[i-1][j-1]-1
        mat[i][j] = max(diag, mat[i-1][j]-1, mat[i][j-1]-1)
a1=[]
a2=[]        

while n>0 or m>0: 
    vert = mat[n-1][m]-1
    hor = mat[n][m-1]-1  
    if x[n-1] == y[m-1]:
        diag1 = mat[n-1][m-1]+1
    else:
        diag1 = mat[n-1][m-1]-1
    if mat[n][m] == vert:
        a1.append(x[n-1])
        a2.append("_")
        n -= 1
    elif mat[n][m] == hor:
        a1.append("_")
        a2.append(y[m-1])
        m -= 1
    elif mat[n][m] == diag1:
        a1.append(x[n-1])
        a2.append(y[m-1])
        n -= 1
        m -= 1

CPU times: user 47.5 s, sys: 908 ms, total: 48.4 s
Wall time: 48.5 s


Процесс отработал за `48.5 секунд`

# 2. "Стандартные биоинформатические" библиотеки

## 2.1 Теоретическая посылка и загрузка пакетов

Считается, что со многими библиотеками (особенно написанными на Си для повышения производительности) PyPy работает крайне неоптимально. Это приводит к проигрышу в скорости по сравнению со стандартным интерпретатором питона. Проверим.

In [1]:
import os
import sys
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

## 2.2 Осмысленный пример

Здесь используется один из скрпитов [семестрового проекта](https://github.com/Asklepiad/BI_project_2022.git). Этот скрипт осуществляет некоторые пертурбации с табличными данными.

In [18]:
%%time
source_path = "/home/asklepiad/bioinf/start_codons/BI_project_2022"
folder_name = "V_campbellii"

### Orto rows with singletons, but without paralog-containing rows

# Uploading and modificating orto rows tables
orto_rows = pd.read_csv(f"{source_path}/{folder_name}/data/{folder_name}.proteinortho.tsv", sep="\t") # Uploading dataframe with orto rows
orto_rows = orto_rows.rename(columns = {"# Species": "Species"})
orto_rows["ortologus_row"] = orto_rows.index + 1 # Creating ortorows numbers

# Uploading first csv-table and creating a new column in it
orto_rows_list = orto_rows.index
df1 = pd.read_csv(f"{source_path}/{folder_name}/data/First_table.csv")

#Filling orto_row column (sounds like an oxymoron)
assemblies = orto_rows.columns[3:-1]
row_assigner = pd.melt(orto_rows, id_vars=["ortologus_row"], value_vars=assemblies)
row_assigner = row_assigner.query("value != '*'")
row_assigner["value"] = row_assigner["value"].str[:14]
row_assigner
row_assigner.drop(["variable"], axis="columns", inplace=True)
row_assigner.columns = ["ortologus_row", "id"]
df1 = df1.merge(row_assigner, on="id")

# Number of the paralogs
print("Number of the paralogs =", sum(orto_rows.query("Genes > Species").Genes) - sum(orto_rows.query("Genes > Species").Species))

# Creating a subset without pararows
pararows_numbers = orto_rows.query("Genes > Species").ortologus_row
df1 = df1.query("ortologus_row not in @pararows_numbers").query("ortologus_row != 0")

# Compiling data about start-codons of ortologus rows
start_codon_per_row = df1.groupby("ortologus_row", as_index=False).agg({"start_codone": ".".join})
start_codon_per_row["start_codone"]
orr_start_list = []
for number in tqdm(range(len(start_codon_per_row))):
    orr_start_list.append(start_codon_per_row.iloc[number, 1].split("."))
start_codons = pd.DataFrame(
    {
        "ortologus_row": start_codon_per_row["ortologus_row"],
        "start_codons": pd.Series(orr_start_list),
        "ATG": 0.0,
        "GTG": 0.0,
        "TTG": 0.0,
    }
)

# Computing frequencies of exact start-codons
start_codons.sort_values("ortologus_row", inplace=True)
for row in tqdm(range(len(start_codons))):
    freqs = pd.Series(start_codons["start_codons"][row]).value_counts() # If it will be need to visualize percents, use normalize=True in brackets
    rowlength = len(start_codons["start_codons"][row])
    if "ATG" in freqs:
        start_codons["ATG"][row] = freqs["ATG"]
    else:
        start_codons["ATG"][row] = 0
    if "GTG" in freqs:
        start_codons["GTG"][row] = freqs["GTG"]
    else:
        start_codons["GTG"][row] = 0
    if "TTG" in freqs:
        start_codons["TTG"][row] = freqs["TTG"]
    else:
        start_codons["TTG"][row] = 0

# Computing uniformity of start-codon per ortologus row
start_codons["uniformity"] = "NA"
for row in range(len(start_codons)):
    if len(set(start_codons.iloc[row, 1])) == 1:
        start_codons.iloc[row, 5] = "same"
    else:
        start_codons.iloc[row, 5] = "different"

# Constructing table withh all data about the ortologus rows (including pararows)
start_codons2 = start_codons.merge(orto_rows, on="ortologus_row", how="outer")

# Creating a table, combining data about gene and its start-codone
strain_gene_row = start_codons2[["Species", "Genes", "ortologus_row", "uniformity", "ATG", "GTG", "TTG"]]   # Other codons deleted
summary_rows = df1.merge(strain_gene_row, on="ortologus_row")

# Assigning cog to orto_row
row_cog = row_assigner.merge(df1[["id", "cog"]], on="id").groupby("ortologus_row").agg({"cog": "max"})
start_codons2 = start_codons2.merge(row_cog, on="ortologus_row")

# Adding organism_name to datasets
summary_rows["organism"] = folder_name
start_codons2["organism"] = folder_name

uniq_cog = start_codons2[["ortologus_row", "ATG", "GTG", "TTG", "uniformity", "Species", "Genes", "cog", "organism"]]

Number of the paralogs = 2719


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 12356/12356 [00:08<00:00, 1538.01it/s]


CPU times: user 10.8 s, sys: 288 ms, total: 11.1 s
Wall time: 11.2 s


Время исполнения - `11.2 секунды`

# 3. Потомки встроенных типов

## 3.1 Теоретический блок и базовый пример.

В CPython и в PyPy по-разному реализовано поведение при добавлении атрибутов извне. CPython предпочитает ориентироваться запись атрибутов извне, а PyPy - на метод `__getitem__`.

У меня нет мыслей, как это можно использовать. Пока ощущение только, что это нужно иметь в виду, дабы не попасться на различиях.

Пример взят из [документации PyPy](https://doc.pypy.org/en/latest/cpython_differences.html#subclasses-of-built-in-types).

In [7]:
class D(dict):
    def __getitem__(self, key):
        if key == 'print':
            return print
        return "PyPy way"

class A(object):
    pass

a = A()
a.__dict__ = D()
a.foo = "CPython way"
print(a.foo)

CPython way
